
# 🕵️ Day 13 — Credit Card Fraud Detection (Advanced Hybrid)

**Goal:** Detect fraudulent transactions with a **hybrid pipeline**:

1) **Unsupervised anomaly scores**  

   • Isolation Forest (outlierness)  

   • Autoencoder (reconstruction error)  

2) **Supervised classifier** (XGBoost) on an enriched feature set  

3) **Imbalance handling** with SMOTE  

4) **Threshold optimization** for **cost-sensitive** business goals  

5) **Evaluation** via Confusion Matrix, ROC, **Precision–Recall**, and **SHAP** explanations

**Dataset:** Kaggle “Credit Card Fraud Detection” — 284,807 transactions, **0.17% fraud**.


## 🔧 1) Setup & Imports

In [ ]:

# If needed on Kaggle, uncomment:
# !pip install -q optuna imbalanced-learn shap xgboost tensorflow

import os, gc, json, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (confusion_matrix, classification_report, roc_auc_score,
                             precision_recall_curve, roc_curve, auc)
from sklearn.ensemble import IsolationForest

from imblearn.over_sampling import SMOTE

import xgboost as xgb
import optuna
import shap
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

RNG = 42
np.random.seed(RNG)
tf.random.set_seed(RNG)
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (6,4)


## 📥 2) Load Data

In [ ]:

CANDIDATES = [
    "/kaggle/input/creditcardfraud/creditcard.csv",
    "/kaggle/input/credit-card-fraud-detection/creditcard.csv",
    "creditcard.csv"
]
data_path = next((p for p in CANDIDATES if os.path.exists(p)), None)
if data_path is None:
    raise SystemExit("❌ Dataset not found. Add the Kaggle dataset ‘mlg-ulb/creditcardfraud’ to the notebook.")

df = pd.read_csv(data_path)
print(df.shape)
df.head()


## 🔎 3) Basic EDA & Target Imbalance

In [ ]:

print(df["Class"].value_counts().rename({0:"Not Fraud", 1:"Fraud"}))
print("Fraud ratio:", df["Class"].mean())

fig, ax = plt.subplots(1,2, figsize=(10,4))
sns.countplot(x="Class", data=df, ax=ax[0])
ax[0].set_title("Target Distribution")

sns.histplot(df["Amount"], bins=50, ax=ax[1])
ax[1].set_title("Amount Distribution")
plt.tight_layout(); plt.show()


## ✂️ 4) Train/Test Split & Scaling

In [ ]:

# Features: V1..V28 + Amount (drop Time)
X = df.drop(columns=["Class", "Time"])
y = df["Class"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RNG
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("Train:", X_train.shape, "Test:", X_test.shape)


## 🌲 5) Unsupervised Signal 1 — Isolation Forest

In [ ]:

iso = IsolationForest(
    n_estimators=300, contamination="auto",
    random_state=RNG, n_jobs=-1
)
iso.fit(X_train_scaled)

# Decision function: higher = more normal. Convert to anomaly score.
train_iso_raw = -iso.decision_function(X_train_scaled)
test_iso_raw  = -iso.decision_function(X_test_scaled)

def minmax01(a):
    lo, hi = np.min(a), np.max(a)
    return (a - lo) / (hi - lo + 1e-9)

train_iso = minmax01(train_iso_raw)
test_iso  = minmax01(test_iso_raw)

pd.Series(train_iso).describe()


## 🤖 6) Unsupervised Signal 2 — Autoencoder

In [ ]:

# Train the autoencoder only on NON-FRAUD class (learn "normal" patterns)
X_train_norm = X_train_scaled[y_train == 0]

input_dim = X_train_norm.shape[1]
encoding_dim = 16  # bottleneck

ae = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(32, activation="relu"),
    layers.Dense(encoding_dim, activation="relu", name="bottleneck"),
    layers.Dense(32, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(input_dim, activation=None)
])

ae.compile(optimizer="adam", loss="mse")
history = ae.fit(
    X_train_norm, X_train_norm,
    epochs=20, batch_size=512,
    validation_split=0.1,
    verbose=0
)

plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Autoencoder Training Loss"); plt.legend(); plt.show()

def recon_err(model, X):
    X_hat = model.predict(X, verbose=0)
    return np.mean((X - X_hat)**2, axis=1)

train_recon = recon_err(ae, X_train_scaled)
test_recon  = recon_err(ae, X_test_scaled)

train_recon = minmax01(train_recon)
test_recon  = minmax01(test_recon)

pd.Series(train_recon).describe()


## 🧱 7) Build Enriched Feature Set for Classifier

In [ ]:

X_train_enriched = np.hstack([X_train_scaled, train_iso.reshape(-1,1), train_recon.reshape(-1,1)])
X_test_enriched  = np.hstack([X_test_scaled,  test_iso.reshape(-1,1),  test_recon.reshape(-1,1)])

enriched_cols = list(X_train.columns) + ["IForest_Anomaly", "AE_ReconErr"]
print("Enriched feature count:", X_train_enriched.shape[1])


## ⚖️ 8) Handle Imbalance with SMOTE

In [ ]:

sm = SMOTE(random_state=RNG, k_neighbors=5, n_jobs=-1)
X_train_bal, y_train_bal = sm.fit_resample(X_train_enriched, y_train)
print("Before:", np.bincount(y_train))
print("After: ", np.bincount(y_train_bal))


## 🎯 9) XGBoost with Optuna (optimize ROC-AUC)

In [ ]:

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 400, 1400),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 10.0, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
        "eval_metric": "auc",
        "random_state": RNG,
        "n_jobs": -1,
        "tree_method": "hist"
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RNG)
    aucs = []
    for tr_idx, va_idx in skf.split(X_train_bal, y_train_bal):
        X_tr, X_va = X_train_bal[tr_idx], X_train_bal[va_idx]
        y_tr, y_va = y_train_bal[tr_idx], y_train_bal[va_idx]
        clf = xgb.XGBClassifier(**params)
        clf.fit(X_tr, y_tr, eval_set=[(X_va, y_va)], verbose=False, early_stopping_rounds=50)
        va_proba = clf.predict_proba(X_va)[:,1]
        aucs.append(roc_auc_score(y_va, va_proba))
    return float(np.mean(aucs))

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=False)

print("Best AUC:", study.best_value)
print("Best params:", json.dumps(study.best_params, indent=2))
best_params = study.best_params


## ✅ 10) Train Best Model & Evaluate

In [ ]:

best_params.update(dict(eval_metric="auc", random_state=RNG, n_jobs=-1, tree_method="hist"))
clf = xgb.XGBClassifier(**best_params)
clf.fit(X_train_bal, y_train_bal, eval_set=[(X_test_enriched, y_test)], verbose=False)

proba = clf.predict_proba(X_test_enriched)[:,1]
roc = roc_auc_score(y_test, proba)
print(f"Test ROC-AUC: {roc:.4f}")

# Default threshold 0.5
pred_05 = (proba >= 0.5).astype(int)
cm = confusion_matrix(y_test, pred_05)
print("Confusion Matrix @0.5:\n", cm)
print(classification_report(y_test, pred_05, digits=4))


## 💰 11) Threshold Optimization (Cost-Sensitive)

In [ ]:

# Business costs: FN (missed fraud) >> FP (manual review)
C_FP = 1    # cost of investigating a legit transaction
C_FN = 10   # cost of missing an actual fraud

def find_best_threshold(y_true, y_proba, c_fp=C_FP, c_fn=C_FN):
    thresholds = np.linspace(0.01, 0.99, 99)
    best = {"thr":0.5, "cost":np.inf, "pr":0, "rc":0}
    for t in thresholds:
        pred = (y_proba >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
        cost = fp*c_fp + fn*c_fn
        pr = tp / (tp + fp + 1e-9)
        rc = tp / (tp + fn + 1e-9)
        if cost < best["cost"]:
            best = {"thr":t, "cost":cost, "pr":pr, "rc":rc}
    return best

best = find_best_threshold(y_test, proba)
best


## 📉 12) Final Evaluation @ Optimized Threshold + Curves

In [ ]:

thr = best["thr"]
pred_opt = (proba >= thr).astype(int)
tn, fp, fn, tp = confusion_matrix(y_test, pred_opt).ravel()

print(f"Chosen threshold: {thr:.2f}")
print("Confusion Matrix @thr={:.2f}:\n".format(thr), np.array([[tn, fp],[fn, tp]]))
print(classification_report(y_test, pred_opt, digits=4))
print(f"ROC-AUC: {roc_auc_score(y_test, proba):.4f}")

# Curves
prec, rec, pr_thr = precision_recall_curve(y_test, proba)
plt.plot(rec, prec); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision–Recall Curve"); plt.show()

fpr, tpr, roc_thr = roc_curve(y_test, proba)
plt.plot(fpr, tpr, label=f"AUC={auc(fpr,tpr):.3f}")
plt.plot([0,1],[0,1],'--',c='gray'); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend(); plt.title("ROC Curve"); plt.show()


## 🧠 13) SHAP Explainability (Top Features)

In [ ]:

# SHAP on a sample for speed
explainer = shap.TreeExplainer(clf)
sample_idx = np.random.choice(len(X_test_enriched), size=min(3000, len(X_test_enriched)), replace=False)
shap_values = explainer.shap_values(X_test_enriched[sample_idx])

feature_names = enriched_cols
shap.summary_plot(shap_values, X_test_enriched[sample_idx], feature_names=feature_names, show=False)
plt.title("SHAP Summary — Top Drivers of Fraud"); plt.show()



## 🧠 Key Takeaways
- **Hybrid works**: Unsupervised signals (Isolation Forest + Autoencoder) improved the supervised classifier.
- **Imbalance matters**: SMOTE helped the classifier learn minority patterns.
- **Cost-sensitive threshold** beats accuracy: we optimized the cut-off to **reduce missed fraud** (FN) at acceptable FP review cost.
- **Explainability** (SHAP) shows which signals push a transaction towards *fraud* (e.g., anomaly scores, Amount, specific PCA components).

## 💼 Business Interpretation
- **False Negatives** are expensive → tune threshold to **maximize recall** within operational FP limits.
- Use model outputs to **prioritize manual review** queues.
- Continuously retrain with **recent fraud patterns** (concept drift).


## 💾 15) Save Model Artifacts

In [ ]:

import joblib, json

joblib.dump(scaler, "scaler.joblib")
joblib.dump(iso, "isolation_forest.joblib")
ae.save("autoencoder.h5")
joblib.dump(clf, "xgb_hybrid.joblib")

with open("feature_names.json","w") as f:
    json.dump(feature_names, f)

print("Saved: scaler.joblib, isolation_forest.joblib, autoencoder.h5, xgb_hybrid.joblib, feature_names.json")


## 🔮 16) Inference Helper

In [ ]:

def fraud_inference(raw_row_dict, thr_override=None):
    """raw_row_dict: { 'V1':..., 'V2':..., ..., 'V28':..., 'Amount':... }"""
    cols = list(X.columns)  # original order
    x_raw = pd.DataFrame([raw_row_dict], columns=cols)
    xs = scaler.transform(x_raw)
    iso_score = minmax01(-iso.decision_function(xs)).reshape(-1,1)
    # reuse AE reconstruction calculation
    ae_err = minmax01(np.mean((xs - ae.predict(xs, verbose=0))**2, axis=1)).reshape(-1,1)
    xe = np.hstack([xs, iso_score, ae_err])

    p = clf.predict_proba(xe)[:,1][0]
    thr = best["thr"] if thr_override is None else thr_override
    lab = int(p >= thr)
    return float(p), lab

# Demo on a real test row
demo = X_test.iloc[0].to_dict()
fraud_inference(demo)
